In [1]:
# Import Library
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import requests
import pandas as pd
import re

In [2]:
# ngetes response server web nya bisa di scraping atau ngga (target: Response[200])
url = "https://karirhub.kemnaker.go.id/"
page = requests.get(url)
page

<Response [200]>

In [3]:
# Setting browser
chrome_options = Options()
chrome_options.add_argument("--window-size=1280,960")

# Ganti path dengan lokasi dari chromedriver.exe
driver = webdriver.Chrome(r"C:\Users\ddeel\Local\KerjaKerjaKerja\chromedriver.exe", chrome_options=chrome_options)
driver.get(url)
sleep(3)

<ipython-input-3-45a9242a4d63>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\ddeel\Local\KerjaKerjaKerja\chromedriver.exe", chrome_options=chrome_options)
<ipython-input-3-45a9242a4d63>:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(r"C:\Users\ddeel\Local\KerjaKerjaKerja\chromedriver.exe", chrome_options=chrome_options)


In [4]:
driver.find_element_by_link_text("Perusahaan").click()

<ipython-input-4-ca38a5ed9dc5>:1: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("Perusahaan").click()


In [5]:
links = []
# Scraping link page dari tiap company nya
for i in range(1,112): # kalau scrapnya mau dikit", range nya dikurangin aja
    url="https://karirhub.kemnaker.go.id/companies?page={}".format(i)
    driver.get(url)
    sleep(5)
    linknodes = driver.find_elements(By.XPATH, "//*[@class='company-upper-text']//a")
    for node in linknodes:
        link = node.get_attribute("href")
        links.append(link)
    sleep(2)

In [6]:
print("Jumlah company :",len(links))

Jumlah company : 998


In [9]:
q1 = links[0:250]
q2 = links[250:500]
q3 = links[500:750]
q4 = links[750:998]

In [12]:
# Buat list untuk menampung hasil scraping
company_names = []
descriptions = []
industries = []
addresses = []
locations = []
phones = []
emails = []
websites = []

In [26]:
# Fungsi buat scraping company info nya
# WARNING! will take a lot of time for running (tergantung internet speed & jumlah links)
def scrap(taut):
    for i in taut:
        url = i
        driver.get(url)
        sleep(7) # Bagian ini kalau internetnya cepet bisa diganti jadi 5 detik aja
        try:
            nama = WebDriverWait(driver, 5).until(EC.presence_of_element_located((
                    By.CSS_SELECTOR, "h2[class='company-name']"))).text
            company_names.append(nama)
        except:
            pass
        sleep(1)
        try:
            desc = WebDriverWait(driver, 5).until(EC.presence_of_element_located((
                    By.CSS_SELECTOR, "p[class='text-muted profile-description']"))).text
            descriptions.append(desc)
        except:
            pass
            descriptions.append("")
        sleep(1)
        try:
            ind = WebDriverWait(driver, 5).until(EC.presence_of_element_located((
                    By.CSS_SELECTOR, "label[class='company-business']"))).text
            industries.append(ind)
        except:
            pass
            industries.append("")
        sleep(1)
        try:
            alamat = WebDriverWait(driver, 5).until(EC.presence_of_element_located((
                    By.CSS_SELECTOR, "div[class='profile-detail-location-container']>p"))).text
            addresses.append(alamat)
        except:
            pass
            addresses.append("")
        sleep(1)
        try:
            lok = WebDriverWait(driver, 5).until(EC.presence_of_element_located((
                    By.CSS_SELECTOR, "label[class='company-location text-muted']"))).text
            locations.append(lok)
        except:
            pass
            locations.append("")
        sleep(1)
        kontak = WebDriverWait(driver, 5).until(EC.presence_of_element_located((
                    By.CSS_SELECTOR, "div[class='d-flex flex-column flex-md-row align-items-start align-items-md-center']"))).text
        try:
            phone = re.split('\n', kontak)[0].strip()
            phones.append(phone)
        except:
            pass
            phones.append("")
        sleep(1)
        try:
            email = re.split('\n', kontak)[1].strip()
            emails.append(email)
        except:
            pass
            emails.append("")
        sleep(1)
        try:
            website = re.split('\n', kontak)[2].strip()
            websites.append(website)
        except:
            pass
            websites.append("")
        sleep(1)

In [14]:
# Estimated running time : 1 hour
scrap(q1)

In [27]:
scrap(q2)

In [33]:
scrap(q3)

In [72]:
scrap(q4)

In [82]:
# input hasil scraping ke dataframe pandas
df = pd.DataFrame()
df['Company Name'] = company_names
df['Description'] = descriptions
df['Industry'] = industries
df['Address'] = addresses
df['Location'] = locations
df['Phone'] = phones
df['E-mail'] = emails
df['Website'] = websites

In [83]:
df.to_excel('output.xlsx')

C:\Users\ddeel\anaconda3\lib\site-packages\pandas\compat\_optional.py:123: UserWarning: Pandas requires version '1.0.2' or newer of 'xlsxwriter' (version '0.7.3' currently installed).
  warnings.warn(msg, UserWarning)
